In [16]:
import pandas as pd

# 1) Função que trata cada célula da coluna 'valor'
def parse_val(v):
    # mantém NaN
    if pd.isna(v):
        return v
    # se já vier número, devolve como está
    if isinstance(v, (int, float)):
        return v
    # senão é string: limpa símbolo e espaços
    s = str(v).replace("R$", "").replace(" ", "")
    # se tiver tanto ponto quanto vírgula, o ponto é separador de milhar
    if "." in s and "," in s:
        s = s.replace(".", "")
    # agora troca a vírgula decimal
    s = s.replace(",", ".")
    return float(s)

# 2) Leitura selecionando só A:E e aplicando o parse ao carregar
df = pd.read_excel(
    "Financas.xlsx",
    usecols="A:E",
    engine="openpyxl",
    converters={"valor": parse_val},
)

# 3) Descarta linhas sem data (os blocos de Saldo/Dívidas etc)
df = df.dropna(subset=["data"])

# 4) (Opcional) garante datetime
df["data"] = pd.to_datetime(df["data"], dayfirst=True)

# 5) Confirma
print(df.head())


        data     categoria      E/D    valor           descrição
0 2025-01-09  Clash Champs  Entrada  3124.55    pagamento do mes
1 2025-01-09          Casa    Saída   700.00                 Mae
2 2025-01-09       Jessica    Saída   150.00  peguei emprestado 
3 2025-01-09      Divorcio    Saída   220.00            divorcio
4 2025-01-09      Academia    Saída   132.90       Treino fisico


In [ ]:
import pandas as pd

# --- 1) Supondo que você já tenha feito isso antes: ---
# df = pd.read_excel(..., usecols="A:E", engine="openpyxl", converters={"valor": parse_val})
# df = df.dropna(subset=["data"])
# df["data"] = pd.to_datetime(df["data"], dayfirst=True)

# --- 2) Filtra somente Abril de 2025 ---
mask_abril = (
    (df["data"].dt.year  == 2025) &
    (df["data"].dt.month == 4)
)
abril = df.loc[mask_abril]

# --- 3) Soma entradas (ganhos) e saídas (gastos) ---
total_entradas = abril.loc[abril["E/D"] == "Entrada", "valor"].sum()
total_saidas   = abril.loc[abril["E/D"] == "Saida",   "valor"].sum()

# --- 4) Exibe o resultado ---
print("📅 Resumo de Abril/2025")
print(f"  • Total Ganhos (Entradas): R$ {total_entradas:,.2f}")
print(f"  • Total Gastos  (Saídas) : R$ {total_saidas:,.2f}")
print(f"  • Saldo do Mês            : R$ {total_entradas - total_saidas:,.2f}")


In [20]:
import pandas as pd
import unicodedata

# 1) Converte corretamente “3.124,55” ou “700,00” em float
def parse_val(v):
    if pd.isna(v): 
        return 0.0
    s = str(v).replace("R$", "").replace(" ", "")
    if "." in s and "," in s:
        s = s.replace(".", "")
    s = s.replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return 0.0

# 2) Remove acentos e lower-case
def normalize(s: str) -> str:
    s = str(s)
    nk = unicodedata.normalize("NFD", s)
    no_acc = "".join(ch for ch in nk if unicodedata.category(ch) != "Mn")
    return no_acc.lower().strip()

# 3) Leitura e limpeza inicial
df = pd.read_excel(
    "Financas.xlsx",
    usecols="A:E",
    engine="openpyxl",
    converters={"valor": parse_val}
).dropna(subset=["data"])
df["data"] = pd.to_datetime(df["data"], dayfirst=True)

# 4) Normaliza o tipo (entrada/saida)
df["tipo"] = df["E/D"].apply(normalize)

# (Opcional: debug)  
# print("Tipos únicos em E/D:", df["tipo"].unique())

# 5) Filtra Abril de 2025
mask = (df["data"].dt.year == 2025) & (df["data"].dt.month == 4)
abril = df.loc[mask]

# 6) Soma entradas e saídas
total_entradas = abril.loc[abril["tipo"] == "entrada", "valor"].sum()
total_saidas   = abril.loc[abril["tipo"] == "saida",   "valor"].sum()

# 7) Resultado
print("📅 Resumo de Abril/2025")
print(f"  • Total Ganhos (Entrada): R$ {total_entradas:,.2f}")
print(f"  • Total Gastos (Saída)  : R$ {total_saidas:,.2f}")
print(f"  • Saldo do Mês           : R$ {total_entradas - total_saidas:,.2f}")


📅 Resumo de Abril/2025
  • Total Ganhos (Entrada): R$ 4,567.84
  • Total Gastos (Saída)  : R$ 4,356.08
  • Saldo do Mês           : R$ 211.76


In [22]:
import pandas as pd
import unicodedata

# 1) Função para converter corretamente valores “3.124,55” ou “700,00” em floats
def parse_val(v):
    if pd.isna(v):
        return 0.0
    s = str(v).replace("R$", "").replace(" ", "")
    if "." in s and "," in s:
        s = s.replace(".", "")
    s = s.replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return 0.0

# 2) Função para normalizar “Entrada”/“Saída” (removendo acentos e lower-case)
def normalize(s: str) -> str:
    nk = unicodedata.normalize("NFD", str(s))
    no_acc = "".join(ch for ch in nk if unicodedata.category(ch) != "Mn")
    return no_acc.lower().strip()

# 3) Carrega apenas colunas A–E e aplica parse_val em “valor”
df = pd.read_excel(
    "Financas.xlsx",
    usecols="A:E",
    engine="openpyxl",
    converters={"valor": parse_val}
).dropna(subset=["data"])

# 4) Converte data e normaliza o tipo
df["data"] = pd.to_datetime(df["data"], dayfirst=True)
df["tipo"] = df["E/D"].apply(normalize)

# 5) Filtra apenas as Saídas de Março/2025 e soma
mask = (
    (df["data"].dt.year  == 2025) &
    (df["data"].dt.month == 3)   &
    (df["tipo"]           == "saida")
)
total_saidas_marco = df.loc[mask, "valor"].sum()

# 6) Exibe o total
print(f"💸 Total de Gastos (Saídas) em Março/2025: R$ {total_saidas_marco:,.2f}")


💸 Total de Gastos (Saídas) em Março/2025: R$ 4,454.33


In [23]:
# 1) Monta a máscara para Maio/2025 e tipo 'saida'
mask_maio = (
    (df["data"].dt.year  == 2025) &
    (df["data"].dt.month == 5)   &
    (df["tipo"]           == "saida")
)

# 2) Calcula a soma das saídas
total_saidas_maio = df.loc[mask_maio, "valor"].sum()

# 3) Exibe o resultado
print(f"💸 Total de Gastos (Saídas) em Maio/2025: R$ {total_saidas_maio:,.2f}")

💸 Total de Gastos (Saídas) em Maio/2025: R$ 3,329.95
